# Lab 3.4: Non-linear registration

In this lab, you will perform a Demons based non-linear registration using SimpleITK.

First of all, you will need to import the modules below. Some of them are in the folder 'L3.3_utilities' that you have already downloaded for the previous lab. They include useful classes and functions which are freely available in the GitHub repository of the [Insight Software Consortium](https://github.com/InsightSoftwareConsortium).

In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from ipywidgets import interact, fixed

# from Insight Software Consortium Github
sys.path.append('./L3.3_utilities')
import registration_utilities as ru
import registration_callbacks as rc
from __future__ import print_function
%run ./L3.3_utilities/popi_utilities_setup.py

## Loading and visualizing the data

The data you are going to work with are 4D (3D space + time) thoracic-abdominal CT images from the [Point-validated Pixel-based Breathing Thorax (POPI)](https://www.creatis.insa-lyon.fr/rio/popi-model?action=show&redirect=popi) model.<br>
In particular, you will need to load:
- 'images', i.e. a list of temporal CT volumes (on five different time steps).
- 'masks', i.e. a list of masks segmenting air, body and lung on each  of the above-described CT volumes. In particular, you are interested in the segmentation of the lung, whose label in the mask is equal to 2.

Use the function _append_ to store each of the four time steps (labeled in their filenames as 00, 20, 40, 60 and 80) in the lists 'image' and 'masks'.<br>
Then, you can visualize the images by using the function _interact_, which allows you to go through different time steps and different coronal slices. As you can see, the volume of the lung is changing in the different time steps, because of the respiration of the subject.

In [4]:
# initialization of the lists
images = []
masks = []

# TODO: label of the lungs in the mask
lung_label = 2

# TODO: read the data
for i in range(0,8,2):
    
    image_file_name = './L3.3_utilities/4DCT-MetaImage/{0}0-P.nii.gz'.format(i)
    mask_file_name = './L3.3_utilities/4DMask-MetaImage/{0}0-air-body-lungs.nii.gz'.format(i)
    
    # TODO: append current file to the lists
    images.append(sitk.ReadImage(image_file_name, sitk.sitkFloat32))
    masks.append(sitk.ReadImage(mask_file_name))
       
# visualization    
interact(display_coronal_with_overlay, temporal_slice=(0,len(images)-1), 
         coronal_slice = (0, images[0].GetSize()[1]-1), 
         images = fixed(images), masks = fixed(masks), 
         label=fixed(lung_label), window_min = fixed(-1024), window_max=fixed(976));

interactive(children=(IntSlider(value=1, description='temporal_slice', max=3), IntSlider(value=179, description='coronal_slice', max=359), Output()), _dom_classes=('widget-interact',))

## Registration

Your task now consists in implementing a function 'demons_registration' which performs Demons-based non-linear registration between two images ('fixed_image' and 'moving_image'), which need to be provided as first two input parameters.<br>
You should then add a third input parameters ('fixed_image_mask'), which represents a certain mask of interest defined in the space of the fixed image. The default value of 'fixed_image_mask' should be 'None', in case the user is not providing any input mask. Otherwise, if a mask is given, the similarity metric of the registration should be evaluated only using points inside the mask.

In [8]:
def demons_registration(fix_img, mov_img, fix_img_mask=None): # TODO: provide input parameters
    
    # TODO: initialize the registration
    reg_method = sitk.ImageRegistrationMethod()

    # TODO: create initial identity transformation.
    transform_to_displacement_field_filter = sitk.TransformToDisplacementFieldFilter()
    transform_to_displacement_field_filter.SetReferenceImage(fix_img)
    initial_transform = sitk.DisplacementFieldTransform(transform_to_displacement_field_filter.Execute(sitk.Transform()))
    
    # TODO: regularization. The update field refers to fluid regularization; the total field to elastic regularization.
    initial_transform.SetSmoothingGaussianOnUpdate(varianceForUpdateField=0.3, varianceForTotalField=0.3) 
    
    # TODO: set the initial transformation
    reg_method.SetInitialTransform(initial_transform)
    
    # TODO: use the function 'SetMetricAsDemons' to be able to perform Demons registration.
    # Be aware that you will need to provide a parameter (the intensity difference threshold) as input:
    # during the registration, intensities are considered to be equal if their difference is less than the given threshold.
    reg_method.SetMetricAsDemons(intensityDifferenceThreshold = 0.001)
    
    # TODO: evaluate the metrics only in the mask, if provided as an input
    if fix_img_mask is not None:
        reg_method.SetMetricFixedMask(fix_img_mask)  
    
    # TODO: set a linear interpolator
    reg_method.SetInterpolator(sitk.sitkLinear)
    
    # TODO: set a gradient descent optimizer
    reg_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=10, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    reg_method.SetOptimizerScalesFromPhysicalShift()
        
    return reg_method.Execute(sitk.Cast(fix_img, sitk.sitkFloat32), 
                                              sitk.Cast(mov_img, sitk.sitkFloat32))    

Once you have implemented the 'demons_registration' function, you can select two time points (between 0 and 4). The two CT scans acquired in these two time points will determine your fixed and moving image.<br>
Then, you can perform Demons registration on the two images. Remember to provide the appropriate input mask (only for the lung label!) stored in the list 'masks'.

In [16]:
# TODO: select two indices (one for each time point)
fixed_image_index = 0
moving_image_index = 3

# TODO: perform the registration
transformation = demons_registration(fix_img = images[fixed_image_index], 
                                     mov_img = images[moving_image_index],
                                     fix_img_mask = (masks[fixed_image_index] == lung_label)
                                     )

Once the registration has finished running, you can estimate its quality both visually and quantitively by using the segmentation masks.<br>
First of all, apply the transformation derived as an output of demons_registration to the mask of the moving image.

In [17]:
# transfer the segmentation via the estimated transformation
transformed_labels = sitk.Resample(masks[moving_image_index],
                                   images[fixed_image_index],
                                   transformation, 
                                   sitk.sitkNearestNeighbor,
                                   0.0, 
                                   masks[moving_image_index].GetPixelID())

Once you got the transformed labels, you can visually assess the quality of the segmentation by judging from the image whether the transformed lung mask is actually matching the lungs in the fixed image or not.<br>
In the figure below, you can see both the original mask (mask index 0) of the moving image and the registered one (mask index 1). Did the registration succeed in properly deforming the lungs?

In [18]:
# visualize
segmentations_before_and_after = [masks[moving_image_index], transformed_labels]
interact(display_coronal_with_label_maps_overlay, coronal_slice = (0, images[0].GetSize()[1]-1),
         mask_index=(0,len(segmentations_before_and_after)-1),
         image = fixed(images[fixed_image_index]), masks = fixed(segmentations_before_and_after), 
         label=fixed(lung_label), window_min = fixed(-1024), window_max=fixed(976));

interactive(children=(IntSlider(value=179, description='coronal_slice', max=359), IntSlider(value=0, description='mask_index', max=1), Output()), _dom_classes=('widget-interact',))

By using the derived transformed labels, the quality of the registration can also be assessed quantitavely. In particular, you can use the __dice coefficient__, which is a metric having values between 0 and 1 and can be used to compare the degree of overlap between the "ground-truth" (i.e. the original lung mask of the fixed image) and the registered mask (i.e. the lung segmentation in 'transformed_labels'). You can read more about the dice coefficient in its [Wikipedia page](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient).<br>
Is the dice coefficient increasing _after_ the registration with respect to _before_ the registration?

In [23]:
# TODO: get the ground truth mask, the mask before the registration and the mask after the registration
ground_truth = masks[fixed_image_index] == lung_label
before_reg = masks[moving_image_index] == lung_label
after_reg = transformed_labels == lung_label

# compute the dice coefficient
label_overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
label_overlap_measures_filter.Execute(ground_truth, before_reg)
print("Dice coefficient before registration: {:.2f}".format(label_overlap_measures_filter.GetDiceCoefficient()))
label_overlap_measures_filter.Execute(ground_truth, after_reg)
print("Dice coefficient after registration: {:.2f}".format(label_overlap_measures_filter.GetDiceCoefficient()))

Dice coefficient before registration: 0.93
Dice coefficient after registration: 0.95


Try to test different parameters inside the 'demons_registration' function to test whether you can further improve your result.